In [ ]:
import csv
import pandas as pd
import os
import tensorflow as tf
import time
import gc
import numpy as np
from scipy.stats import spearmanr

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/Research/Adversarial_Attacks_Model-VS-Human/datasets/clickme_test_1000.tfrecords' ./

In [ ]:
!pip install -q harmonization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.8/656.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from harmonization.models import (load_ViT_B16, load_ResNet50,
                                  load_VGG16, load_EfficientNetB0,
                                  load_tiny_ConvNeXT, load_tiny_MaxViT,
                                  load_LeViT_small,
                                  preprocess_input)
# model = load_ResNet50()
# model_name = "resnet50v2_harmonized"

# model = load_VGG16()
# model_name = "vgg16_harmonized"

# model = load_ViT_B16()
# model_name = "vit_b16_harmonized"

# model = load_EfficientNetB0()
# model_name = "efficientnet_b0_harmonized"

model = load_tiny_ConvNeXT()
model_name = "convnext_tiny_harmonized"

# model = load_tiny_MaxViT()
# model_name = "maxvit_tiny_harmonized"

# Bad Performance !!!
# model = load_LeViT_small()
# model_name = "levit_small_harmonized"


In [ ]:
class Dataset:
    def __init__(self, data_path):
        self.data_path = data_path
        
        self.AUTO = tf.data.AUTOTUNE

        self._feature_description = {
            "image"       : tf.io.FixedLenFeature([], tf.string, default_value=''),
            "heatmap"     : tf.io.FixedLenFeature([], tf.string, default_value=''),
            "label"       : tf.io.FixedLenFeature([], tf.int64, default_value=0),
        }

    def parse_prototype(self, prototype, training=False):
        data    = tf.io.parse_single_example(prototype, self._feature_description)

        image   = tf.io.decode_raw(data['image'], tf.float32)
        image   = tf.reshape(image, (224, 224, 3))
        image   = tf.cast(image, tf.float32)

        heatmap = tf.io.decode_raw(data['heatmap'], tf.float32)
        heatmap = tf.reshape(heatmap, (224, 224, 1))

        label   = tf.cast(data['label'], tf.int32)
        label   = tf.one_hot(label, 1_000)

        return image, heatmap, label

    def get_dataset(self, batch_size, training=False):
        deterministic_order = tf.data.Options()
        deterministic_order.experimental_deterministic = True

        dataset = tf.data.TFRecordDataset([self.data_path], num_parallel_reads=self.AUTO)
        dataset = dataset.with_options(deterministic_order) 
        
        dataset = dataset.map(self.parse_prototype, num_parallel_calls=self.AUTO)
        
        dataset = dataset.batch(batch_size, drop_remainder=True)
        dataset = dataset.prefetch(self.AUTO)

        return dataset
        
datapath = "/content/clickme_test_1000.tfrecords"
data = Dataset(datapath)

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False,)

# @tf.function
def grads_step(xt, model, labels):
    with tf.GradientTape() as tape:
        tape.watch(xt)
        outputs = model(xt)
        # print(outputs.shape, labels.shape)     
        cost = tf.reduce_sum(loss(outputs, labels)) # (1, 1000) one-hot vector
    grads = tape.gradient(cost, xt)
    return grads

def copy(x):
  return tf.cast(np.array(x).copy(), tf.float32)

class Attack:
    def __init__(self):
        pass
    
    def l2(self, x):
        norm = tf.reduce_sum(tf.square(x), (1,2,3)) # batch_size, H, W, C
        norm = tf.sqrt(norm + 1e-4)
        return norm[:,None,None,None] # batch_size, 1, 1, 1

    def l2_pgd_attack(self, model, images, labels, eps, alpha=10/255, iters=5):   
        x0 = tf.cast(copy(images), tf.float32)
        labels = tf.cast(labels, tf.float32)[None, :] # (1, 1000)
        xt = copy(x0)   

        for i in range(iters) :        
            # print(xt.shape, labels.shape)
            grads = grads_step(xt, model, labels)
            x_next = xt + grads / self.l2(grads) * alpha
            
            # project the current point on the l2(x0, epsilon) ball :)
            delta = x_next - x0
            sigma = self.l2(delta) # norm
            x_next = x_next + (delta / sigma) * eps
            
            # ready for the next step
            xt = x_next

        return xt


In [ ]:
def execute_attack(model, img, target, eps, alpha):
    # Setup an attack
    attack_obj = Attack()
    perturbed_img = attack_obj.l2_pgd_attack(model, img, target, eps=eps, alpha=0.5, iters=3)

    # Re-classify the perturbed image
    outputs = model(perturbed_img)
    pred = tf.argmax(outputs, -1)
    
    return perturbed_img, pred

def write_csv_all(record, path):
    header = ['model', 'img', 'label', 'pred', 'eps', 'l2', 'linf', 'spearman']
    file_exists = os.path.isfile(path)

    with open(path, mode='a+', newline='') as csv_file:
        writer = csv.writer(csv_file)
        if not file_exists:
            writer.writerow(header)
        writer.writerow(record)

def write_csv_avg(record, path):
    header = [
        'model', 'num_correct', 
        'avg_eps', 'std_eps', 
        'avg_l2', 'std_l2', 
        'avg_linf', 'std_linf', 
        'avg_spearman', 'std_spearman']

    file_exists = os.path.isfile(path)
    with open(path, mode='a+', newline='') as csv_file:
        writer = csv.writer(csv_file)
        if not file_exists:
            writer.writerow(header)
        writer.writerow(record)
        

In [ ]:
# from tensorflow.keras.models import load_model
# model_name = "convnext_tiny_harmonized_5"
# pretrained = "/content/drive/MyDrive/Research/Adversarial_Attacks_Model-VS-Human/Harmonizer zoo/models/"
# model = load_model(pretrained + model_name + ".h5")

# Define paths
case = "L2PGD_0.5_3_harmonized"
folder_path = '/content/drive/MyDrive/Research/Adversarial_Attacks_Model-VS-Human/results/' + case + '/'
results_path_avg = folder_path + case + '.csv'
results_path_all = folder_path + model_name + '.csv'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Created folder: {folder_path}")
else:
    print(f"Folder already exists: {folder_path}")


l2_list, linf_list = [], []
opt_epsilons = []
spearman_scores = []
total_cnt, init_correct, aa_correct = 0, 0, 0

for imgs, hmps, labels in data.get_dataset(2 , False): # image, heatmap, label
    imgs = tf.cast(imgs, tf.float32)
    hmps = tf.cast(hmps, tf.float32)
    imgs = preprocess_input(imgs)
    labels = tf.cast(labels, tf.float32)

    for img, hmp, label in zip(imgs, hmps, labels):
        # Add a batch dimension
        img = img[None, :, :, :] 
        hmp = hmp[None, :, :, :]
        target = tf.argmax(label[:, None]) # tf.Tensor([343], shape=(1,), dtype=int64)

        # Predict
        output = model(img)
        init_pred = tf.argmax(output, axis=-1) # tf.Tensor([343], shape=(1,), dtype=int64)
        # print(label.shape, target, init_pred)

        # If the initial prediction is wrong, just move to the next image
        total_cnt += 1
        print("\rSearching optimal epsilon for image: %s | %s %s" % (str(total_cnt), str(1000), model_name), end=" ")
        if init_pred.numpy()[0] != target.numpy()[0]:
            continue
        init_correct += 1
        
        # Apply first attack
        initial_eps = 10
        perturbed_img, perturbed_pred = execute_attack(model=model, img=img, target=label, eps=initial_eps, alpha=0.5)

        if perturbed_pred.numpy()[0] == target.numpy()[0]: # Assume 10 is the min eps that fools the model
            optimal_eps = initial_eps
        else:
            # key: eps; val: perturbed_img
            info = {} # Only store one key-val pair
            key = None

            initial_eps = 1
            perturbed_img, perturbed_pred = execute_attack(model=model, img=img, target=label, eps=initial_eps, alpha=0.5)

            key = initial_eps
            info[key] = (perturbed_img, perturbed_pred)

            if perturbed_pred.numpy()[0] == target.numpy()[0]: 
                l, r = 1, 10 # search eps between 1 and 10
                threshold = 0.1 
            else:
                l, r = 0.001, 1 # search eps between 0.001 and 1
                threshold = 0.01
            
            while r - l >= threshold:
                eps = l + (r - l) / 2
                # print(eps)
                perturbed_img, perturbed_pred = execute_attack(model=model, img=img, target=label, eps=eps, alpha=0.5)
                if perturbed_pred.numpy()[0] == target.numpy()[0]: 
                    l = eps
                else:
                    r = eps
                    if r != key:
                        del info[key]
                        key = r
                        info[key] = (perturbed_img, perturbed_pred)

            optimal_eps = r
            if r in info:
                perturbed_img, perturbed_pred = info[r]
            else:
                perturbed_img, perturbed_pred = execute_attack(model=model, img=img, target=label, eps=optimal_eps, alpha=0.5)

        opt_epsilons.append(optimal_eps)
        # print(optimal_eps)

        # Store l2, linf
        l2, linf = tf.norm(perturbed_img - img, ord=2).numpy(), tf.norm(perturbed_img - img, ord=np.inf).numpy()
        l2_list.append(l2)
        linf_list.append(linf)

        # Spearman correlation
        mask = np.abs(np.mean(perturbed_img.numpy() - img.numpy(), axis=-1, keepdims=True)) # (1, 1, 224, 224)
        spearman_score, _ = spearmanr(mask.flatten(), hmp.numpy().flatten())
        spearman_scores.append(spearman_score)

        # Save the data into 
        row_data = [
            model_name, str(total_cnt-1), str(target.numpy()[0]), str(perturbed_pred.numpy()[0]), 
            str(round(optimal_eps, 6)), str(l2), str(linf), str(spearman_score)
        ]
        # print(row_data)
        write_csv_all(row_data, results_path_all)
        
print("")


Folder already exists: /content/drive/MyDrive/Research/Adversarial_Attacks_Model-VS-Human/results/L2PGD_0.5_3_harmonized/
Searching optimal epsilon for image: 1000 | 1000 convnext_tiny_harmonized_5 


In [ ]:
# Save data
row_data = [
    model_name, str(init_correct), 
    str(np.mean(opt_epsilons)), str(np.std(opt_epsilons)),
    str(np.mean(l2_list)), str(np.std(l2_list)),
    str(np.mean(linf_list)), str(np.std(linf_list)),
    str(np.mean(spearman_scores)), str(np.std(spearman_scores)),
]
print(row_data)
write_csv_avg(row_data, results_path_avg)

['convnext_tiny_harmonized_5', '889', '6.418002847300338', '3.632960596041218', '19.988203', '10.723778', '1.6975623', '1.1887592', '0.4428087849804497', '0.1776509614698785']
